# Qwen 2.5 3B Counting Benchmark on Google Colab

This notebook benchmarks Qwen/Qwen2.5-3B-Instruct on the counting task (fits comfortably on T4 GPU).

## Setup: Install Dependencies

In [ ]:
!pip install -q transformers accelerate torch tqdm

## Clone Repository and Load Dataset

In [ ]:
!git clone https://github.com/Larsen-Daniel/llm-counting-analysis.git
%cd llm-counting-analysis

## Check GPU Availability

In [ ]:
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## Benchmark Code

In [ ]:
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from typing import List, Dict
from tqdm import tqdm
import re

def load_dataset(file_path: str) -> List[Dict]:
    """Load the dataset from JSONL file."""
    examples = []
    with open(file_path, 'r') as f:
        for line in f:
            examples.append(json.loads(line))
    return examples

def extract_answer(text: str) -> int | None:
    """
    Extract numerical answer from model output.
    Try multiple patterns in order of preference:
    1. Last number in parentheses: (N)
    2. First number on first line
    """
    # First try to find number in parentheses
    matches = re.findall(r'\((\d+)\)', text)
    if matches:
        return int(matches[-1])  # Return the last match

    # Fall back to first number in the output
    first_line = text.strip().split('\n')[0]
    number_match = re.search(r'\d+', first_line)
    if number_match:
        return int(number_match.group(0))

    return None  # No number found

def benchmark_qwen(
    model_name: str,
    dataset: List[Dict],
    max_examples: int = None,
    device: str = "cuda" if torch.cuda.is_available() else "cpu"
) -> Dict:
    """
    Benchmark Qwen model on the counting task.

    Args:
        model_name: HuggingFace model name
        dataset: List of examples
        max_examples: Maximum examples to evaluate
        device: Device to run on

    Returns:
        Dictionary with results
    """
    print(f"\n{'='*80}")
    print(f"Benchmarking: {model_name}")
    print(f"Device: {device}")
    print(f"{'='*80}\n")

    # Load model and tokenizer
    print("Loading model and tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16 if device == "cuda" else torch.float32,
        device_map="auto" if device == "cuda" else None,
        low_cpu_mem_usage=True
    )

    if device == "cpu":
        model = model.to(device)

    model.eval()

    # Prepare dataset
    if max_examples:
        dataset = dataset[:max_examples]

    results = {
        "model_id": model_name,
        "total_examples": len(dataset),
        "correct": 0,
        "incorrect": 0,
        "parse_errors": 0,
        "numerical_errors": 0,
        "api_errors": 0,
        "predictions": []
    }

    print(f"Evaluating on {len(dataset)} examples...")

    for example in tqdm(dataset):
        prompt = example["prompt"]
        true_answer = example["answer"]

        # Generate
        inputs = tokenizer(prompt, return_tensors="pt").to(device)

        with torch.no_grad():
            outputs = model.generate(
                inputs.input_ids,
                max_new_tokens=10,
                do_sample=False,
                pad_token_id=tokenizer.eos_token_id
            )

        generated_text = tokenizer.decode(
            outputs[0][inputs.input_ids.shape[1]:],
            skip_special_tokens=True
        )

        # Extract answer
        predicted_answer = extract_answer(generated_text)

        # Check correctness
        if predicted_answer is None:
            results["parse_errors"] += 1
            is_correct = False
        else:
            is_correct = predicted_answer == true_answer
            if is_correct:
                results["correct"] += 1
            else:
                results["numerical_errors"] += 1
                results["incorrect"] += 1

        # Store prediction
        results["predictions"].append({
            "prompt": prompt,
            "true_answer": true_answer,
            "predicted_answer": predicted_answer,
            "generated_text": generated_text,
            "correct": is_correct
        })

    # Calculate metrics
    results["accuracy"] = results["correct"] / results["total_examples"]
    results["parse_error_rate"] = results["parse_errors"] / results["total_examples"]
    results["numerical_error_rate"] = results["numerical_errors"] / results["total_examples"]
    results["api_error_rate"] = results["api_errors"] / results["total_examples"]

    print(f"\nResults for {model_name}:")
    print(f"  Accuracy: {results['accuracy']:.2%} ({results['correct']}/{results['total_examples']})")
    print(f"  Parse Errors: {results['parse_error_rate']:.2%}")
    print(f"  Numerical Errors: {results['numerical_error_rate']:.2%}")

    return results

## Run Benchmark

In [ ]:
# Load dataset
dataset = load_dataset('data/counting_dataset.jsonl')
print(f"Loaded {len(dataset)} examples")

# Run benchmark on 1000 examples
results = benchmark_qwen(
    model_name="Qwen/Qwen2.5-3B-Instruct",
    dataset=dataset,
    max_examples=1000
)

# Save results
with open('results/qwen_benchmark_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print("\nResults saved to results/qwen_benchmark_results.json")

## Download Results

In [ ]:
from google.colab import files
files.download('results/qwen_benchmark_results.json')

## Mediation Analysis (Optional)

In [ ]:
# Pull latest changes and reload
!git pull origin main

# Import after adding scripts to path
import sys
sys.path.append('scripts')

import importlib
import mediation_utils
importlib.reload(mediation_utils)

# Mediation Analysis
from mediation_utils import generate_minimal_pairs, run_mediation_analysis

print("\nGenerating minimal pairs...")
pairs = generate_minimal_pairs(dataset, n_pairs=200)

print(f"Running mediation analysis on {len(pairs)} pairs...")
mediation_results = run_mediation_analysis(model, tokenizer, pairs, device='cuda')

with open('results/mediation_results_qwen3b.json', 'w') as f:
    json.dump(mediation_results, f, indent=2)

print("\nTop 5 layers by causal effect:")
effects = [(int(i), data['mean_effect']) for i, data in mediation_results['layer_effects'].items()]
effects.sort(key=lambda x: x[1], reverse=True)
for layer, effect in effects[:5]:
    print(f"  Layer {layer}: {effect:.3f}")

files.download('results/mediation_results_qwen3b.json')